<a href="https://colab.research.google.com/github/binibinibini/project2/blob/main/%EB%AA%A8%EB%8D%B8%ED%95%99%EC%8A%B5%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. 필요 라이브러리 설치
!pip install -q transformers[torch] datasets pandas scikit-learn accelerate

# 2. 라이브러리 임포트
import pandas as pd
import torch
import numpy as np
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from google.colab import drive

print("✅ 라이브러리 설치 및 임포트 완료!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.4 MB/s eta 0:00:00
✅ 라이브러리 설치 및 임포트 완료!


In [ ]:
# 1. 구글 드라이브 마운트
try:
    drive.mount('/content/drive')
    print("✅ Google Drive가 성공적으로 마운트되었습니다.")
except Exception as e:
    print(f"❌ Google Drive 마운트 중 오류 발생: {e}")
    raise

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive가 성공적으로 마운트되었습니다.


In [ ]:
def get_positive_integer(prompt_message):
    """
    사용자에게 0을 초과하는 자연수를 입력받는 함수.
    잘못된 입력(음수, 0, 소수, 문자)이 들어오면 다시 입력받도록 반복합니다.
    """
    while True:
        try:
            user_input = input(prompt_message)
            number = int(user_input)

            # 변경된 부분: number가 0보다 큰지 확인합니다.
            if number > 0:
                return number
            else:
                print("⚠️ 오류: 0보다 큰 자연수를 입력해주세요.")
        except ValueError:
            print("⚠️ 오류: 유효한 숫자가 아닙니다. 다시 입력해주세요.")

epochs = get_positive_integer("epoch를 입력하세요: ")

# 2. 파일 경로 설정 (★★사용자 수정 필요★★)
# 학습에 사용할 CSV 파일의 전체 경로를 정확하게 입력해주세요.
csv_file_path = '/content/drive/MyDrive/ITWILL(중간플젝2)/파인튠드 분류 모델/모델 파인튜닝/통합_감정분류완료_최종.csv'

# 파인튜닝된 모델이 저장될 폴더 경로
output_dir = f'/content/drive/MyDrive/ITWILL(중간플젝2)/파인튠드 분류 모델/kote-finetuned-model-{epochs}epoch'

print(f"📂 학습용 CSV 파일 경로: {csv_file_path}")
print(f"💾 모델 저장 폴더: {output_dir}")

epoch를 입력하세요: 10
📂 학습용 CSV 파일 경로: /content/drive/MyDrive/ITWILL(중간플젝2)/파인튠드 분류 모델/모델 파인튜닝/통합_감정분류완료_최종.csv
💾 모델 저장 폴더: /content/drive/MyDrive/ITWILL(중간플젝2)/파인튠드 분류 모델/kote-finetuned-model-10epoch


In [ ]:
# 1. CSV 파일 로드
try:
    # 'utf-8-sig' 인코딩으로 파일을 읽도록 수정
    df = pd.read_csv(csv_file_path, encoding='utf-8-sig')
    print("✅ CSV 파일 로드 성공!")
    print(f"Original data size: {df.shape}")
except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다: {csv_file_path}")
    print("2단계의 'csv_file_path' 변수가 올바르게 설정되었는지 확인해주세요.")
    raise
except UnicodeDecodeError:
    print(f"❌ 인코딩 오류가 발생했습니다. 파일을 'CSV UTF-8' 형식으로 다시 저장한 후 시도해보세요.")
    raise

# 2. 필요한 컬럼 선택 및 전처리
emotion_labels = ['기쁨', '기대', '애정', '슬픔', '분노', '공포']
# '좋아요수'와 '답글수'를 포함하여 컬럼 선택
required_cols = ['댓글내용', '좋아요수', '답글수'] + emotion_labels
df = df[required_cols].copy()

# 3. '좋아요수', '답글수'의 빈 값을 0으로 채우기
df['좋아요수'] = df['좋아요수'].fillna(0).astype(int)
df['답글수'] = df['답글수'].fillna(0).astype(int)

# 4. [수정] 감정 레이블 컬럼의 타입을 `float`으로 지정합니다.
df[emotion_labels] = df[emotion_labels].fillna(0).astype(float)

# 5. 메타데이터를 포함한 새로운 'text' 컬럼 생성
df['text'] = df.apply(
    lambda row: f"[좋아요: {row['좋아요수']}, 답글: {row['답글수']}] {row['댓글내용']}",
    axis=1
)

# 6. 'labels' 컬럼을 다중 레이블 형식으로 생성
df['labels'] = df[emotion_labels].values.tolist()
df = df[['text', 'labels']]

# 텍스트 내용이 없는 행 제거
df.dropna(subset=['text'], inplace=True)

print("\n✅ 데이터 전처리 완료. 레이블 타입을 float으로 변경했습니다.")
print("--- Example of the new text format ---")
print(df.head())

# 7. Pandas DataFrame을 Hugging Face Dataset으로 변환
dataset = Dataset.from_pandas(df)

# 8. 훈련(train) 및 검증(validation) 데이터셋으로 분리 (80:20)
dataset = dataset.train_test_split(test_size=0.2, seed=42)

print("\n✅ Train/validation split complete:")
print(dataset)

✅ CSV 파일 로드 성공!
Original data size: (1939, 14)

✅ 데이터 전처리 완료. 레이블 타입을 float으로 변경했습니다.
--- Example of the new text format ---
                                                text  \
0             [좋아요: 2036, 답글: 21] 등장인물 다 까먹은 사람이면 개추   
1            [좋아요: 1184, 답글: 18] 이시리즈를 젊을때 본다는게 축복이다   
2  [좋아요: 996, 답글: 4] 어른되고도 뭔가저런 상상의 세상에\n설레고 신난건 ...   
3    [좋아요: 778, 답글: 0] 볼때마다 경이로운 상상력과 CG는 감탄만 나온다 ㄹㅇ   
4  [좋아요: 414, 답글: 24] 더워 죽겠는데 전독시같은거나 보면서 짜증나다가 예...   

                           labels  
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  
1  [1.0, 0.0, 1.0, 0.0, 0.0, 0.0]  
2  [1.0, 1.0, 1.0, 0.0, 0.0, 0.0]  
3  [1.0, 0.0, 0.0, 0.0, 0.0, 0.0]  
4  [0.0, 1.0, 0.0, 0.0, 0.0, 0.0]  

✅ Train/validation split complete:
DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 1551
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 388
    })
})


In [ ]:
# 1. 베이스 모델 이름 정의
model_name = "searle-j/kote_for_easygoing_people"

# 2. 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 3. 레이블 정보 정의
labels = emotion_labels
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

# 4. 모델 로드 (다중 레이블 분류용으로 설정)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    problem_type="multi_label_classification",
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True  # 이 부분을 추가하여 크기 불일치 오류를 해결합니다.
)

print("✅ 모델 및 토크나이저 로드 완료!")
print("INFO: ignore_mismatched_sizes=True 로 설정하여, 분류기(classifier) 레이어는 새로 초기화되었습니다.")
print("      이제 이 분류기 레이어를 우리의 데이터에 맞게 파인튜닝하게 됩니다.")

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at searle-j/kote_for_easygoing_people and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([44, 768]) in the checkpoint and torch.Size([6, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([44]) in the checkpoint and torch.Size([6]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ 모델 및 토크나이저 로드 완료!
INFO: ignore_mismatched_sizes=True 로 설정하여, 분류기(classifier) 레이어는 새로 초기화되었습니다.
      이제 이 분류기 레이어를 우리의 데이터에 맞게 파인튜닝하게 됩니다.


In [ ]:
# Define the tokenization function
def preprocess_data(examples):
  # Tokenize the text. The 'text' column now includes metadata.
  tokenized_batch = tokenizer(examples['text'], padding=True, truncation=True, max_length=512)
  # Convert label data type to float32
  tokenized_batch['labels'] = [[float(i) for i in label] for label in examples['labels']]
  return tokenized_batch

# Apply the tokenization function to the entire dataset
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=['text'])

print("✅ Data tokenization complete!")

Map:   0%|          | 0/1551 [00:00<?, ? examples/s]

Map:   0%|          | 0/388 [00:00<?, ? examples/s]

✅ Data tokenization complete!


In [ ]:
# 1. 평가 지표 계산 함수 정의
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Sigmoid를 적용하여 확률값으로 변환
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # 0.5를 기준으로 0 또는 1로 변환
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= 0.5)] = 1
    y_true = labels

    # F1 score (weighted average)
    f1_weighted = f1_score(y_true=y_true, y_pred=y_pred, average='weighted')
    # Accuracy (전체 예측 중 정답 비율)
    accuracy = accuracy_score(y_true, y_pred)

    return {'f1': f1_weighted, 'accuracy': accuracy}

# 2. 학습 인자 (TrainingArguments) 설정
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=epochs,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    # [수정] 인자 이름을 'eval_strategy'로 변경합니다.
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_dir='./logs',
    logging_steps=10,
)

# 3. Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 4. 모델 학습 시작
print("\n🔥 모델 파인튜닝을 시작합니다...")
trainer.train()

# 5. 최종 모델 저장
trainer.save_model(output_dir)
print(f"\n🎉 파인튜닝 완료! 최종 모델이 '{output_dir}'에 저장되었습니다.")

/tmp/ipython-input-1831548495.py:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



🔥 모델 파인튜닝을 시작합니다...


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `ElectraSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.369200,0.348533,0.436351,0.353093
2,0.322600,0.323522,0.570680,0.394330
3,0.228200,0.323781,0.626689,0.438144
4,0.168000,0.330811,0.623908,0.404639
5,0.178000,0.355095,0.620725,0.412371
6,0.150300,0.365809,0.637763,0.427835
7,0.114000,0.387607,0.616340,0.407216
8,0.072900,0.409218,0.606669,0.399485
9,0.073900,0.411795,0.622277,0.414948
10,0.068200,0.414886,0.620176,0.409794


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `ElectraSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `ElectraSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `ElectraSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `ElectraSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/


🎉 파인튜닝 완료! 최종 모델이 '/content/drive/MyDrive/ITWILL(중간플젝2)/파인튠드 분류 모델/kote-finetuned-model-10epoch'에 저장되었습니다.


In [ ]:
from transformers import pipeline

# 이전에 학습된 모델이 저장된 경로를 확인합니다.
# output_dir 변수가 6단계에서 사용한 경로와 동일한지 확인하세요.
print(f"로드할 모델 경로: {output_dir}")

# 파인튜닝된 모델로 추론 파이프라인 생성
try:
    pipe = pipeline("text-classification", model=output_dir, return_all_scores=True)
    print("✅ 추론 파이프라인 로드 완료!")
except OSError:
    print(f"❌ 모델을 찾을 수 없습니다. '{output_dir}' 경로에 모델 파일이 있는지 확인해주세요.")
    # 오류가 발생하면 여기서 실행을 멈춥니다.
    raise

# --- 테스트할 문장 목록 ---
test_cases = [
    {
        "comment_text": "와, 정말 유익한 영상이네요! 감사합니다.",
        "likes": 15,
        "replies": 2,
        "note": "긍정/감사"
    },
    {
        "comment_text": "자꾸 이상한 소리만 하네. GPT 진짜 멍청해지는듯.",
        "likes": 30,
        "replies": 5,
        "note": "분노/불만"
    },
    {
        "comment_text": "이러다 진짜 AI가 인간을 지배하는 세상이 오는 거 아닐까요? 너무 무서워요.",
        "likes": 5,
        "replies": 1,
        "note": "공포/불안"
    },
    {
        "comment_text": "힘들 때 AI한테 위로받는다는 댓글 보니까 마음이 좀 짠하네요...",
        "likes": 45,
        "replies": 3,
        "note": "슬픔/공감"
    },
    {
        "comment_text": "앞으로 AI가 어떻게 발전할지 궁금하고 기대됩니다.",
        "likes": 8,
        "replies": 0,
        "note": "기대/중립"
    },
    {
        "comment_text": "와~ AI 덕분에 이제 댓글도 믿을 수가 없겠네. 정말 대단하다.",
        "likes": 12,
        "replies": 4,
        "note": "모호함/반어법"
    },
    {
        "comment_text": "객관적인 정보 요약은 잘하는데, 조금만 복잡해지면 없는 말을 지어내서 문제임.",
        "likes": 55,
        "replies": 11,
        "note": "비판 (분노/슬픔)"
    }
]

# 각 테스트 케이스에 대해 추론 실행 및 결과 출력
for i, case in enumerate(test_cases):
    # 학습 때 사용한 형식과 동일하게 입력 텍스트를 구성
    input_text = f"[좋아요: {case['likes']}, 답글: {case['replies']}] {case['comment_text']}"

    # 추론 실행
    result = pipe(input_text)

    print("\n" + "="*40)
    print(f"   테스트 케이스 #{i+1} ({case['note']})")
    print("="*40)
    print(f"입력 문장: \"{input_text}\"")
    print("-" * 20)
    print("감정 분석 결과:")

    # 점수가 높은 순으로 정렬하여 출력
    sorted_scores = sorted(result[0], key=lambda x: x['score'], reverse=True)
    for r in sorted_scores:
        print(f"- {r['label']:<4s}: {r['score']:.4f}")

로드할 모델 경로: /content/drive/MyDrive/ITWILL(중간플젝2)/파인튠드 분류 모델/kote-finetuned-model-5epoch


Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `ElectraSelfAttention.forward`.
  return forward_call(*args, **kwargs)


✅ 추론 파이프라인 로드 완료!

   테스트 케이스 #1 (긍정/감사)
입력 문장: "[좋아요: 15, 답글: 2] 와, 정말 유익한 영상이네요! 감사합니다."
--------------------
감정 분석 결과:
- 기쁨  : 0.9270
- 애정  : 0.1517
- 기대  : 0.0654
- 슬픔  : 0.0397
- 분노  : 0.0328
- 공포  : 0.0180

   테스트 케이스 #2 (분노/불만)
입력 문장: "[좋아요: 30, 답글: 5] 자꾸 이상한 소리만 하네. GPT 진짜 멍청해지는듯."
--------------------
감정 분석 결과:
- 분노  : 0.8996
- 슬픔  : 0.1146
- 공포  : 0.0969
- 기대  : 0.0578
- 기쁨  : 0.0354
- 애정  : 0.0194

   테스트 케이스 #3 (공포/불안)
입력 문장: "[좋아요: 5, 답글: 1] 이러다 진짜 AI가 인간을 지배하는 세상이 오는 거 아닐까요? 너무 무서워요."
--------------------
감정 분석 결과:
- 공포  : 0.8560
- 기대  : 0.2223
- 슬픔  : 0.2115
- 분노  : 0.1932
- 애정  : 0.0617
- 기쁨  : 0.0484

   테스트 케이스 #4 (슬픔/공감)
입력 문장: "[좋아요: 45, 답글: 3] 힘들 때 AI한테 위로받는다는 댓글 보니까 마음이 좀 짠하네요..."
--------------------
감정 분석 결과:
- 슬픔  : 0.8815
- 분노  : 0.1303
- 애정  : 0.0851
- 공포  : 0.0602
- 기대  : 0.0315
- 기쁨  : 0.0200

   테스트 케이스 #5 (기대/중립)
입력 문장: "[좋아요: 8, 답글: 0] 앞으로 AI가 어떻게 발전할지 궁금하고 기대됩니다."
--------------------
감정 분석 결과:
- 기대  : 0.7424
- 기쁨  : 0.2016
- 공포  : 0.1205
- 애정  : 0.1049